# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,-22.09,71,20,0.00,SJ,1712449836
1,1,carnarvon,-24.8667,113.6333,25.26,58,0,7.37,AU,1712450016
2,2,swakopmund,-22.6833,14.5333,14.43,81,1,1.34,NaN,1712450016
3,3,utrik,11.2278,169.8474,27.02,75,100,8.87,MH,1712449843
4,4,vorgashor,67.5833,63.9500,-9.55,93,100,7.14,RU,1712449957


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot= city_data_df.hvplot.points("Lng","Lat",tiles="OSM", geo=True, size = "Humidity", color = "City")

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Ideal Weather= A max temperature lower than 30 degrees but higher than 20, Wind speed less than 5 m/s, humidity between 40-60%, cloudiness is less than 30
ideal_weather=city_data_df.loc[(city_data_df["Max Temp"]>=20)&(city_data_df["Max Temp"]<=30)&
                               (city_data_df["Wind Speed"]<=6)&
                               (city_data_df["Humidity"]>=40)&(city_data_df["Humidity"]<=60)& 
                               (city_data_df["Cloudiness"]<=30)]

# Drop any rows with null values
ideal_weather_clean= ideal_weather.dropna()

# Display sample data
ideal_weather_clean


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
119,119,alice springs,-23.7000,133.8833,20.75,40,4,0.51,AU,1712449980
140,140,duba,27.3513,35.6901,23.35,60,0,4.24,SA,1712450039
239,239,garbsen-mitte,52.4266,9.6038,21.10,56,0,3.09,DE,1712450058
312,312,colonia,50.9333,6.9500,21.09,46,0,2.57,DE,1712449896
434,434,armidale,-30.5167,151.6500,20.65,54,0,3.52,AU,1712449865
451,451,guerrero,17.6667,-100.0000,23.77,45,6,1.95,MX,1712450096
474,474,sweetwater,25.7634,-80.3731,25.01,57,0,5.14,US,1712449818


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= city_data_df[["City", "Country","Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= " "

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,longyearbyen,SJ,78.2186,15.6401,71,
1,carnarvon,AU,-24.8667,113.6333,58,
2,swakopmund,NaN,-22.6833,14.5333,81,
3,utrik,MH,11.2278,169.8474,75,
4,vorgashor,RU,67.5833,63.9500,93,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# used https://apidocs.geoapify.com/docs/geocoding/forward-geocoding/#about and instructor code for reference 
# Set parameters to search for a hotel
limit = 20
radius = 10000
categories= "accommodation.hotel"
params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories":categories
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    Latitude = row["Lat"]
    Longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    # params["categories"] = f"accommodation.hotel"
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
longyearbyen - nearest hotel: The Vault
carnarvon - nearest hotel: No hotel found
swakopmund - nearest hotel: Amanpuri Travellers Lodge
utrik - nearest hotel: No hotel found
vorgashor - nearest hotel: София
huarmey - nearest hotel: Hostal Santa Rosa
amravati - nearest hotel: Hotel Hindustan International
varde - nearest hotel: Hotel Arnbjerg
edinburgh of the seven seas - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
west island - nearest hotel: Cocos Village Bungalows
bethel - nearest hotel: Hampton Inn Danbury
puerto maldonado - nearest hotel: Puerto Palmeras
bredasdorp - nearest hotel: Victoria Hotel
saudarkrokur - nearest hotel: Mikligarður
chibougamau - nearest hotel: No hotel found
lauro de freitas - nearest hotel: Malibu Plaza Hotel
hobyo - nearest hotel: Iftin Hotel Hobyo
loukhi - nearest hotel: Северяночка
volchansk - nearest hotel: Гостиница ВМЗ
port lincoln - nearest hotel: Boston Hotel
am timan - nearest hotel: Nimla
puerto na

,City,Country,Lat,Lng,Humidity,Hotel Name
0,longyearbyen,SJ,78.2186,15.6401,71,The Vault
1,carnarvon,AU,-24.8667,113.6333,58,No hotel found
2,swakopmund,NaN,-22.6833,14.5333,81,Amanpuri Travellers Lodge
3,utrik,MH,11.2278,169.8474,75,No hotel found
4,vorgashor,RU,67.5833,63.9500,93,София
...,...,...,...,...,...,...
565,mount gambier,AU,-37.8333,140.7667,79,The Old Mount Gambier Gaol
566,rifle,US,39.5347,-107.7831,69,Comfort Inn & Suites
567,griffith,US,41.5284,-87.4237,73,Quality Inn & Suites Hammond
568,benghazi,LY,32.1167,20.0667,74,ترجاب


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot_hotel= hotel_df.hvplot.points("Lng","Lat", tiles="OSM", hover_cols=["City","Country","Hotel Name"],geo=True)

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)